# Load libraries and function

In [1]:
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import xarray as xr
xr.set_options(display_style="html")

from pathlib import Path
import scipy.io  as sio
import numpy as np
import pandas as pd
from matplotlib import animation as ani, pyplot as plt

import re

from tensorpac.utils import ITC
from tqdm.notebook import tqdm
import logging
import sys
import pickle

def get_project_root() -> Path:
    """Return a base path of current folder, similar to R packege 'here'.
        
    Returns:
        Path: Path object pointing to the parent of the folder where the script is located.
            If your script is `/root/scripts/script.py`, the result will be an absolute path '/root'.
    """
    try:
        return Path(__file__).parent.parent
    except NameError as error:
        # This will give you  root folder of the project if this runned through jupyter notebook
        return Path().joinpath().absolute().parents[0]
    except Exception as exception:
        print(exception)

# Work with xarray

Before I exported ROIS from Brainstorm to one of folders, now I am loading the files into the list. I am  also checking some properties of the imported data (it is imported as a dict structure, I wrote custom function to parse that dict)                                                            

In [4]:
def get_real_names(row_names):

    row_names=  [l.flatten()[0] for l in row_names.flatten()] #flatten
    scout_names = [i.split(" @ ")[0] for i in row_names]
    subject  = [i.split(" @ ")[1].split("/")[0] for i in row_names]
    condition  = [i.split(" @ ")[1].split("/")[1] for i in row_names]
    trial= [i.split(" @ ")[1].split("/")[2] for i in row_names] 
    numbers = [re.findall("\d+", i)[0] for i in trial]
    trials= [int(i) for i in numbers]
    subject_cond = [x.split( "_", 3)[3] for x in condition]

    frame = { 'scout_names' : scout_names, 'subject': subject,'subject_cond':subject_cond, 'trial_str': trial, 'trial': trials  }
    return pd.DataFrame(frame)


In [5]:

folder = '/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG'
files =  [x for x in Path(folder).glob("*.mat")]
#files = files[:1]
files

[PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0412.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0515.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0311.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0315.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0320.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0324.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0327.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0331.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0336.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0340.mat'),
 PosixPath('/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0345.mat'),

In [6]:
Values = []
Descriptions = []

for file in files:
    print(file)
    mat_content = sio.loadmat(file )
    Time = mat_content["Time"]
    Values.append(mat_content["Value"])
    Descriptions.append(get_real_names(mat_content["Description"]))
display(Descriptions[0])
display(Values[0].shape)   
    

/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0412.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0515.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0311.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0315.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0320.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0324.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0327.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0331.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0336.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0340.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0345.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0350.mat
/Volumes/ExtremePro/tDCS/Preprocessed/tDCS_MEG/matrix_scout_210723_0355.mat
/Volumes/Ext

: 

: 

## Create  xarray object

I loading the data from matlab structure here and I am creating a custom structure to hold it using `xarray` package.
The mat file has intertwined trials and ROIS (we could see above the structure in `Descriptions` dataframe.
After reshaping and transpoition the structure has dims `scout names` x `trial` x `times`. 

In [16]:
# Here is simple test how nmpy reshape behave I am playing here with reshaping 
test_array = np.array([[11. ,11. ,11.], [21. ,21. ,21.],[31. ,31. ,31.],[12. ,12. ,12.], [22. ,22. ,22.],[32. ,32. ,32.]])
display(test_array)
number_rois = 3; number_trials  = 2 # 2 triale, 3 rois
reshaped_array = np.reshape(test_array, (-1, number_rois, 3))
display(reshaped_array)

array([[11., 11., 11.],
       [21., 21., 21.],
       [31., 31., 31.],
       [12., 12., 12.],
       [22., 22., 22.],
       [32., 32., 32.]])

array([[[11., 11., 11.],
        [21., 21., 21.],
        [31., 31., 31.]],

       [[12., 12., 12.],
        [22., 22., 22.],
        [32., 32., 32.]]])

In [23]:
# TODO check if row__names contain only expected dat

def create_dataaray(mat_content : dict): 
    """A simple function to  create an DataArray object"""

    #display(mat_content)
    Time = mat_content["Time"]
    result = get_real_names(mat_content["Description"])
    scout_names = list(result["scout_names"].unique())
    trials = list(result["trial"].unique())
    subject = list(result["subject"].unique())[0]
    subject_cond = list(result["subject_cond"].unique())[0]
    attrs={'sfreq': 600, 'subject': subject, 'subject_cond': subject_cond}
    Value = mat_content["Value"]
    reshaped_timeseries = np.reshape(Value, (-1, len(scout_names), len(Time[0])))

    da = xr.DataArray(
    reshaped_timeseries,
    dims=[ 'trial','scout_names' ,'time'],
    coords={
        'trial': trials,
        'scout_names': scout_names,
        'time': Time[0]
    },
    name= subject,
    attrs=attrs)
    
    return da.transpose('scout_names', 'trial', 'time')

Below are some functions that can by applied to xarray object. Ideas was taken from  [Chris Holdgraf  blog](https://predictablynoisy.com/posts/2019/2019-10-22-xarray-neuro/#de-meaning-the-data-with-da-where).

In [1]:
def plot_xarray(temp):
    fig, ax = plt.subplots(figsize=(20, 5))
    n_channels = da['scout_names'].shape[0]
    ax.set_prop_cycle(color=plt.cm.viridis(np.linspace(0, 1, n_channels)))
    temp.mean(dim='trial').plot.line(x='time', hue='scout_names')
    ax.get_legend().remove()

def plot_imshow(data, title ):
    #fig, ax = plt.subplots(figsize=(20, 5))
    #ax.set_prop_cycle(color=plt.cm.viridis(np.linspace(0, 1, n_channels)))
    fig, ax = plt.subplots(figsize=(20,10))
    cs  = ax.imshow(data, interpolation='none', aspect='auto', extent =[-2,2, 0, 154]) 
    cbar = fig.colorbar(cs)
    plt.title(title)

    plt.show()
    
def filter_array(da, flow = 2,fhigh = 43):
    da_filter = xr.apply_ufunc(
        mne.filter.filter_data, da,
       kwargs=dict(
           sfreq=da.sfreq,
           l_freq=flow,
           h_freq=fhigh,
           n_jobs = -1
       )
    ) 
    return da_filter

def plot_itpc(temp):
    fig, ax = plt.subplots(figsize=(20, 5))
    n_channels = temp['scout_names'].shape[0]
    ax.set_prop_cycle(color=plt.cm.viridis(np.linspace(0, 1, n_channels)))
    temp.plot.line(x='time', hue='scout_names')
    ax.get_legend().remove()
    
def highpass_filter(da, flow = 1.,fhigh = None):
    da_filter = xr.apply_ufunc(
        mne.filter.filter_data, da,
       kwargs=dict(l_freq=flow, h_freq=fhigh, fir_design='firwin', verbose=False, n_jobs = -1, sfreq = 600.))
    return da_filter



Now quick iteration how my data looks  like. In the iteration I am trying to plot the raw data and the high pass filered data too.


## Compute xarray through iteration

In [2]:
folder = '/home/daniel/Projects/MEG ASSR Analysis/meg-analysis-clean/data/preprocessed/extracted_ROIS_timeseries/tDCS_MEG'
files =  [x for x in Path(folder).glob("*.mat")]
files = files[:1]

datasets = []
datasets_filtered = []

for file in files:
    print(file)
    mat_content = sio.loadmat(file )
    da = create_dataaray(mat_content)
    datasets.append(da)
    # Plot data filtered around 40hz
    #da_filter= highpass_filter(da, flow = 1.,fhigh = None)
    da_filter = filter_array(da, flow = 39.,fhigh = 41)
    datasets_filtered.append(da)
    # Compute and plot ITPC for regions
    

NameError: name 'Path' is not defined

## Plot xarrays

In [27]:
for i, (da, da_filtered) in enumerate(zip(datasets, datasets_filtered)):
    plot_xarray(da)
    plot_xarray(da_filter) 
    plot_imshow(da_filter.mean(dim='trial').values)
    #plot_imshow(output_itpc
    plt.show()

# Compute ITPC

>Both measures were estimated for the entire epoch. The investiga-
tion of baseline differences was performed focusing on the [−1 s to 0 s]
time window. Once ruled out baseline differences, both measures Baseline time-window was selected
to avoid proximity to the sound and edge effects. Both z-transformed
measures were averaged in the 300–700 ms time-window, as brain
activity reaches a steady state after about 200 ms of 40 Hz entrainment



## Apply functions to xarray data 

I am ploting here the data to check them overall, I will Compute and plot the Inter-Trial Coherence (ITC) using [TensorPac library](https://etiennecmb.github.io/tensorpac/auto_examples/misc/plot_itc.html). Tensorpac provides two ways for extracting phase and amplitude :
-  Using filtering followed by Hilbert transform.
-  Using wavelets.


In [28]:
def itpc_2d(da):
    """Perform a ITPC on (n_channels, n_times) array."""
    
    def itpc_to_xarray(data_itpc, da):
        """A simple function to convert an itpc numpy array  to DataArray"""
        itpc_array = xr.DataArray(
        data_itpc,
        dims=['scout_names', 'time'],
        coords={
            'time': da.time,
            'scout_names': da.scout_names,
        },
        name=da.name,
        attrs= da.attrs
        )
        return itpc_array

    array = da.values.copy()
    
    lista_itc = [] 
    sf = 600.       # sampling frequency
    f_min = 39      # minimum sine frequency
    f_max = 41      # maximum sine frequency
    #edges = 10  # remove 10 points to remove edge effects due to filtering
    cycle = 6   # number of cycles to use to extract the phase
    logging.disable(sys.maxsize) # Python 3
    for ii, channel in enumerate(tqdm(array)):    
        itc = ITC(channel, sf, f_pha=[f_min, f_max], edges=None, cycle=cycle, n_jobs=-1);
        lista_itc.append(itc.itc)
    logging.disable(logging.NOTSET)
    data_itpc = np.stack(lista_itc, axis=0)
    return itpc_to_xarray(data_itpc, da)

## Plot ITPC

## Make a xarray form output

https://etiennecmb.github.io/tensorpac/generated/tensorpac.utils.ITC.html

# Combining information  together

In [29]:
folder = '/media/daniel/ExtremePro1/tDCS/Preprocessed/tDCS_MEG'
files =  [x for x in Path(folder).glob("*.mat")]
#datasets = []
itpc_list = []

for file in files:
    print(file)
    mat_content = sio.loadmat(file )
    da = create_dataaray(mat_content)
    #datasets.append(da)
    # Plot data filtered around 40hz
    #da_filter= highpass_filter(da, flow = 1.,fhigh = None)
    #da_filter = filter_array(da, flow = 39.,fhigh = 41)
    #datasets_filtered.append(da)
    # Compute and plot ITPC for regions
    output_itpc = itpc_2d(da)
    itpc_list.append(output_itpc)   

In [31]:
root = get_project_root()
filename  = Path.joinpath(root,"data/share/ITPC", "list_itpc.pickle")

abspath = Path(filename).absolute()
with open(str(abspath), 'wb') as f:
    pickle.dump(itpc_list, f)

## Plotting results together

In [32]:
# ADD subject name to plot 
for i, item_itpc  in enumerate(itpc_list):
    #plot_imshow(da_filter.mean(dim='trial').values)
    plot_imshow(item_itpc.values, item_itpc.attrs["subject"] )